# VIX Skew Analysis

## Setup & Imports

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import brentq
from IPython.display import display
from datetime import timedelta

plt.style.use('seaborn-v0_8')

VALUATION_DATE = pd.Timestamp("2025-10-13")

def black_price(forward, strike, rate, time_to_expiry, vol, option='call'):
    if time_to_expiry <= 0:
        intrinsic = max(forward - strike, 0.0) if option == 'call' else max(strike - forward, 0.0)
        return np.exp(-rate * time_to_expiry) * intrinsic
    vol = max(vol, 1e-6)
    std_dev = vol * np.sqrt(time_to_expiry)
    d1 = (np.log(forward / strike) + 0.5 * std_dev ** 2) / std_dev
    d2 = d1 - std_dev
    df = np.exp(-rate * time_to_expiry)
    if option == 'call':
        return df * (forward * norm.cdf(d1) - strike * norm.cdf(d2))
    return df * (strike * norm.cdf(-d2) - forward * norm.cdf(-d1))

def black_implied_vol(price, forward, strike, rate, time_to_expiry, option='call'):
    if price <= 0:
        return np.nan
    intrinsic = 0.0
    if option == 'call':
        intrinsic = np.exp(-rate * time_to_expiry) * max(forward - strike, 0.0)
    else:
        intrinsic = np.exp(-rate * time_to_expiry) * max(strike - forward, 0.0)
    if price < intrinsic:
        price = intrinsic

    def objective(vol):
        return black_price(forward, strike, rate, time_to_expiry, vol, option) - price

    try:
        return brentq(objective, 1e-4, 5.0, maxiter=200)
    except ValueError:
        return np.nan

def black_delta(forward, strike, rate, time_to_expiry, vol, option='call'):
    if time_to_expiry <= 0 or vol <= 0:
        if option == 'call':
            return 1.0 if forward > strike else 0.0
        return -1.0 if forward < strike else 0.0
    std_dev = vol * np.sqrt(time_to_expiry)
    d1 = (np.log(forward / strike) + 0.5 * std_dev ** 2) / std_dev
    df = np.exp(-rate * time_to_expiry)
    if option == 'call':
        return df * norm.cdf(d1)
    return -df * norm.cdf(-d1)


## Load Data

In [ ]:

chain_path = 'data/VIX_65d_Options_ChainData.csv'
underlying_path = 'data/VIX_65d_Options_UnderlyingData.csv'

chain_raw = pd.read_csv(chain_path)
underlying = pd.read_csv(underlying_path)

chain_raw['Call Mid'] = (chain_raw['Call Bid'] + chain_raw['Call Ask']) / 2
chain_raw['Put Mid'] = (chain_raw['Put Bid'] + chain_raw['Put Ask']) / 2

merged = chain_raw.merge(underlying, on='T', how='left', suffixes=('', '_underlying'))

unique_tenors = merged['T'].unique()
tenor_map = dict(zip(sorted(unique_tenors), ['UXZ5', 'UXF6']))
merged['Expiry'] = merged['T'].map(tenor_map)

tenor_calendar = []
for tenor in sorted(unique_tenors):
    days = int(round(tenor * 365))
    expiry_date = VALUATION_DATE + timedelta(days=days)
    tenor_calendar.append({'T': tenor, 'Calendar Days': days, 'Expiry (Approx)': expiry_date.date()})

pd.DataFrame(tenor_calendar)


## Compute Forwards

In [ ]:

expiry_results = {}
summary_rows = []

for expiry, group in merged.groupby('Expiry'):
    grp = group.sort_values('Strike').reset_index(drop=True).copy()
    rate = grp['r'].iloc[0]
    future_hint = grp['Future'].iloc[0]
    tenor = grp['T'].iloc[0]

    grp['Forward (Parity)'] = grp['Strike'] + np.exp(rate * tenor) * (grp['Call Mid'] - grp['Put Mid'])

    near_atm = grp.iloc[(np.abs(grp['Strike'] - future_hint)).argsort()[:5]]
    forward_level = near_atm['Forward (Parity)'].mean()
    grp['Forward Level'] = forward_level

    summary_rows.append({'Expiry': expiry,
                         'Time to Expiry': tenor,
                         'Forward (Future)': future_hint,
                         'Forward (Parity Avg)': forward_level})
    expiry_results[expiry] = grp

summary_df = pd.DataFrame(summary_rows)
summary_df


## Implied Volatilities

In [ ]:

for expiry, grp in expiry_results.items():
    rate = grp['r'].iloc[0]
    tenor = grp['T'].iloc[0]
    forward_level = grp['Forward Level'].iloc[0]

    grp['Bloomberg Call IV'] = grp['Call IVM'] / 100
    grp['Bloomberg Put IV'] = grp['Put IVM'] / 100
    grp['Mid Call IV'] = grp.apply(
        lambda row: black_implied_vol(row['Call Mid'], forward_level, row['Strike'], rate, tenor, 'call'), axis=1)
    grp['Mid Put IV'] = grp.apply(
        lambda row: black_implied_vol(row['Put Mid'], forward_level, row['Strike'], rate, tenor, 'put'), axis=1)

    atm_idx = (grp['Strike'] - forward_level).abs().idxmin()
    atmf_vol = np.nanmean([grp.loc[atm_idx, 'Mid Call IV'], grp.loc[atm_idx, 'Mid Put IV']])
    grp['ATMF Vol'] = atmf_vol

    expiry_results[expiry] = grp

expiry_results['UXZ5'][['Strike', 'Call Mid', 'Put Mid', 'Mid Call IV', 'Mid Put IV']].head()


## Delta Calculations

In [ ]:

for expiry, grp in expiry_results.items():
    rate = grp['r'].iloc[0]
    tenor = grp['T'].iloc[0]
    forward_level = grp['Forward Level'].iloc[0]

    grp['Call Delta'] = grp.apply(
        lambda row: black_delta(forward_level, row['Strike'], rate, tenor, row['Mid Call IV'], 'call'), axis=1)
    grp['Put Delta'] = grp.apply(
        lambda row: black_delta(forward_level, row['Strike'], rate, tenor, row['Mid Put IV'], 'put'), axis=1)

    atm_idx = (grp['Strike'] - forward_level).abs().idxmin()
    grp['ATM Call Delta'] = grp.loc[atm_idx, 'Call Delta']

    grp['Log Moneyness'] = np.log(grp['Strike'] / forward_level)
    denom = grp['ATMF Vol'] * np.sqrt(tenor)
    grp['Z-Score'] = np.where(denom > 0, grp['Log Moneyness'] / denom, np.nan)
    grp['Center Delta'] = grp['Call Delta'] - 0.5
    grp['Local Delta'] = grp['Call Delta']

    expiry_results[expiry] = grp

expiry_results['UXF6'][['Strike', 'Call Delta', 'Put Delta', 'Center Delta', 'Local Delta']].head()


## Skew Plotting

In [ ]:

    axis_config = [
        ('Strike', 'Strike'),
        ('Log Moneyness', 'Log Moneyness'),
        ('Z-Score', 'Z-Score'),
        ('Center Delta', 'Center Delta'),
        ('Local Delta', 'Local Delta'),
    ]

    for expiry in ['UXZ5', 'UXF6']:
        grp = expiry_results[expiry].sort_values('Strike').copy()
        rate = grp['r'].iloc[0]
        tenor = grp['T'].iloc[0]
        forward_level = grp['Forward Level'].iloc[0]
        atmf_vol = grp['ATMF Vol'].iloc[0]
        atm_delta = grp['ATM Call Delta'].iloc[0]

        display(pd.DataFrame({'Metric': ['Forward (Parity Avg)', 'ATMF Vol', 'ATM Call Delta'],
                              'Value': [forward_level, atmf_vol, atm_delta]}))

        for axis_label, axis_col in axis_config:
            fig, ax = plt.subplots(figsize=(9, 5))
            x = grp[axis_col]
            ax.plot(x, grp['Bloomberg Call IV'] * 100, label='Bloomberg Call IV', linestyle='--', color='tab:blue')
            ax.plot(x, grp['Mid Call IV'] * 100, label='Mid Call IV', color='tab:orange')
            ax.plot(x, grp['Mid Put IV'] * 100, label='Mid Put IV', color='tab:green')
            ax.axhline(atmf_vol * 100, color='gray', linestyle=':', label='ATMF Vol')
            if axis_col in ['Log Moneyness', 'Center Delta']:
                ax.axvline(0.0, color='gray', linestyle=':')
            if axis_col == 'Strike':
                ax.axvline(forward_level, color='red', linestyle=':', label='Forward')
            if axis_col == 'Local Delta':
                ax.axvline(atm_delta, color='purple', linestyle=':', label='ATM Delta')
            ax.set_title(f"{expiry} Skew vs. {axis_label}")
            ax.set_xlabel(axis_label)
            ax.set_ylabel('Implied Volatility (%)')
            ax.legend()
            ax.grid(True, alpha=0.3)
            ax.annotate(f"Forward: {forward_level:.2f}
ATMF Vol: {atmf_vol:.2%}",
                        xy=(0.02, 0.95), xycoords='axes fraction', ha='left', va='top',
                        bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
            plt.show()



### Skew Interpretation

- **UXZ5:** The near-dated expiry shows pronounced downside skew, with implied volatility rising as strikes fall (negative log-moneyness) and put deltas increase in magnitude. The mid-derived volatilities sit slightly below Bloomberg levels around the forward but converge in the wings, suggesting tight mid quotes near the money.
- **UXF6:** The longer expiry exhibits a flatter skew through the center delta axis, though downside strikes still command higher volatilities. Mid-call and mid-put volatilities align closely except deep OTM puts, highlighting wider put spreads in the back month.


## Summary Interpretation

In [ ]:

comparison = []
for expiry, grp in expiry_results.items():
    comparison.append({
        'Expiry': expiry,
        'Forward (Parity Avg)': grp['Forward Level'].iloc[0],
        'Future Reference': grp['Future'].iloc[0],
        'ATMF Vol': grp['ATMF Vol'].iloc[0],
        'ATM Call Delta': grp['ATM Call Delta'].iloc[0]
    })
summary_table = pd.DataFrame(comparison)
summary_table



The parity-derived forwards track the quoted futures within a few basis points for both expiries, validating the time-to-expiry alignment with the 13 October 2025 valuation date. UXZ5 carries the lower forward level and a higher ATMF volatility, reflecting nearer-term event risk and a steeper downside skew. UXF6 forwards sit higher with slightly lower ATMF volatility, and its skew profile is comparatively muted, indicating calmer expectations further along the curve.
